In [1]:
import numpy as np
import pandas as pd
import datetime
import chess
import chess.syzygy

from lib import generate_table
from pathlib import Path
import datetime
import pandas
import os


In [4]:
%%time
df= generate_table(100000)


0  of  100000 , time:  22:31:20.458877
8/8/8/8/8/8/8/8 w - - 0 1
51190
 2    23080
 0    22906
-2     5202
-1        2
Name: wdl, dtype: int64
CPU times: user 12.4 s, sys: 35.9 ms, total: 12.5 s
Wall time: 12.5 s


In [14]:
n_iter = 100000
board_start = chess.Board("K1k5/P7/8/8/8/8/8/8 w - - 0 0")
board_start0 = chess.Board("8/8/8/8/8/8/8/8 w - - 0 0")

l_boards = []
l_wdl = []
pieces = ["K", "R", "P", "k", "r", "p"]
pc_num = len(pieces)
pieces_types = [6, 4, 1, 6, 4, 1]
pieces_color = [True, True, True, False, False, False]
pieces_chess = [chess.Piece.from_symbol(s) for s in pieces]
i_min = [0, 0, 1, 0, 0, 1]
i_max = [8, 8, 7, 8, 8, 7]
j_min = [0, 0, 0, 0, 0, 0]
j_max = [8, 8, 8, 8, 8, 8]

l_boards = []
l_wdl = []
board = board_start0.copy()
i_counter = 0

with chess.syzygy.open_tablebase(r"syzygy") as tablebases:
    for ii in range(n_iter):
        if i_counter % 100000 == 0:
            print(i_counter, " of ", n_iter, ", time: ", datetime.datetime.now().strftime("%H:%M:%S.%f"))
            print(board.fen())
        board = board_start0.copy()
        i_board = []
        i_board_i_j = []
        turn = True
        board.turn = turn
        i_board.append(turn)
        i_piece = 0
        for i_piece in range(6):
            i = np.random.randint(i_min[i_piece], i_max[i_piece])
            j = np.random.randint(j_min[i_piece], j_max[i_piece])
            board.set_piece_at(i * 8 + j, pieces_chess[i_piece])
            i_board.append(i * 8 + j)
            i_board_i_j.append(i)
            i_board_i_j.append(j)
        if len(i_board) != len(set(i_board)):
            i_counter = i_counter + 1
            continue
        if board.is_valid():
            l_boards.append(i_board)
            l_wdl.append(tablebases.probe_wdl(board))  # *(turn*2-1))
        i_counter = i_counter + 1

df1 = pd.DataFrame(l_boards)
df1.columns = ["move", "K", "R", "P", "k", "r", "p"]
df1["wdl"] = l_wdl
df2 = df1.drop_duplicates()
print(len(df2.index))
print(df2.wdl.value_counts())
df2 = df2.reset_index(drop=True)
 


0  of  100000 , time:  22:42:23.531897
8/8/8/8/8/8/8/8 w - - 0 1
51184
 2    23187
 0    22870
-2     5127
Name: wdl, dtype: int64


In [5]:
%load_ext line_profiler


In [67]:
%lprun -f generate_table generate_table(10000)


0  of  10000 , time:  14:31:49.873185
8/8/8/8/8/8/8/8 w - - 0 1
5173
 2    2334
 0    2302
-2     537
Name: wdl, dtype: int64


In [18]:
tablebases=chess.syzygy.open_tablebase(r"syzygy")

In [16]:
if i_counter % 100000 == 0:
    print(i_counter, " of ", n_iter, ", time: ", datetime.datetime.now().strftime("%H:%M:%S.%f"))
    print(board.fen())
board = board_start0.copy()
i_board = []
i_board_i_j = []
turn = True
board.turn = turn
i_board.append(turn)
i_piece = 0
for i_piece in range(6):
    i = np.random.randint(i_min[i_piece], i_max[i_piece])
    j = np.random.randint(j_min[i_piece], j_max[i_piece])
    board.set_piece_at(i * 8 + j, pieces_chess[i_piece])
    i_board.append(i * 8 + j)
    i_board_i_j.append(i)
    i_board_i_j.append(j)
board.is_valid()

100000  of  100000 , time:  22:44:03.429315
8/8/8/2K5/1R4p1/4P3/8/k2r4 w - - 0 1


True

In [33]:
%lprun -f tablebases.probe_wdl tablebases.probe_wdl(board)

In [41]:
%lprun -f tablebases.probe_ab tablebases.probe_ab(board,-2,2)

In [21]:
%lprun -f tablebases.probe_wdl_table tablebases.probe_wdl_table(board)

In [23]:
key = chess.syzygy.calc_key(board)

In [24]:
table = tablebases.wdl[key]

In [25]:
tablebases._bump_lru(table)

In [26]:
%lprun -f table.probe_wdl_table table.probe_wdl_table(board)

In [32]:
%lprun -f table._probe_wdl_table table._probe_wdl_table(board)

In [31]:
%lprun -f chess.syzygy.calc_key chess.syzygy.calc_key(board)

In [29]:
key

'KRPvKRP'

In [39]:
%%time
t=board.occupied_co[not board.turn]

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 4.53 µs


In [40]:
%%time
board.generate_legal_moves(to_mask=t)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 8.58 µs


<generator object Board.generate_legal_moves at 0x7f4003945c78>

In [46]:
%%time
run(t)

CPU times: user 35 µs, sys: 0 ns, total: 35 µs
Wall time: 37.9 µs


In [43]:
def run(t):
    list(board.generate_legal_moves(to_mask=t))

In [50]:
%lprun -f board.generate_legal_moves board.generate_legal_moves(to_mask=t)

In [51]:
%lprun -f board.generate_legal_moves run(t)

In [42]:
board.generate_legal_moves(to_mask=t)

<generator object Board.generate_legal_moves at 0x7f4003945e58>

In [52]:
%lprun -f board.generate_pseudo_legal_moves list(board.generate_pseudo_legal_moves(chess.BB_ALL, t))

In [57]:
from_mask=chess.BB_ALL
to_mask=t
our_pieces = board.occupied_co[board.turn]
non_pawns = our_pieces & ~board.pawns & from_mask
t1=list(chess.scan_reversed(non_pawns))[0]

In [58]:
%lprun -f board.attacks_mask board.attacks_mask(t1)

In [59]:
t1

40

In [60]:
%%time
board.attacks_mask(t1)

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 10.5 µs


72618349279904000

In [61]:
SQUARES = [
    A1, B1, C1, D1, E1, F1, G1, H1,
    A2, B2, C2, D2, E2, F2, G2, H2,
    A3, B3, C3, D3, E3, F3, G3, H3,
    A4, B4, C4, D4, E4, F4, G4, H4,
    A5, B5, C5, D5, E5, F5, G5, H5,
    A6, B6, C6, D6, E6, F6, G6, H6,
    A7, B7, C7, D7, E7, F7, G7, H7,
    A8, B8, C8, D8, E8, F8, G8, H8,
] = range(64)

In [64]:
SQUARES

range(0, 64)

In [65]:
BB_SQUARES = [
    BB_A1, BB_B1, BB_C1, BB_D1, BB_E1, BB_F1, BB_G1, BB_H1,
    BB_A2, BB_B2, BB_C2, BB_D2, BB_E2, BB_F2, BB_G2, BB_H2,
    BB_A3, BB_B3, BB_C3, BB_D3, BB_E3, BB_F3, BB_G3, BB_H3,
    BB_A4, BB_B4, BB_C4, BB_D4, BB_E4, BB_F4, BB_G4, BB_H4,
    BB_A5, BB_B5, BB_C5, BB_D5, BB_E5, BB_F5, BB_G5, BB_H5,
    BB_A6, BB_B6, BB_C6, BB_D6, BB_E6, BB_F6, BB_G6, BB_H6,
    BB_A7, BB_B7, BB_C7, BB_D7, BB_E7, BB_F7, BB_G7, BB_H7,
    BB_A8, BB_B8, BB_C8, BB_D8, BB_E8, BB_F8, BB_G8, BB_H8
] = [1 << sq for sq in SQUARES]

In [66]:
BB_SQUARES

[1,
 2,
 4,
 8,
 16,
 32,
 64,
 128,
 256,
 512,
 1024,
 2048,
 4096,
 8192,
 16384,
 32768,
 65536,
 131072,
 262144,
 524288,
 1048576,
 2097152,
 4194304,
 8388608,
 16777216,
 33554432,
 67108864,
 134217728,
 268435456,
 536870912,
 1073741824,
 2147483648,
 4294967296,
 8589934592,
 17179869184,
 34359738368,
 68719476736,
 137438953472,
 274877906944,
 549755813888,
 1099511627776,
 2199023255552,
 4398046511104,
 8796093022208,
 17592186044416,
 35184372088832,
 70368744177664,
 140737488355328,
 281474976710656,
 562949953421312,
 1125899906842624,
 2251799813685248,
 4503599627370496,
 9007199254740992,
 18014398509481984,
 36028797018963968,
 72057594037927936,
 144115188075855872,
 288230376151711744,
 576460752303423488,
 1152921504606846976,
 2305843009213693952,
 4611686018427387904,
 9223372036854775808]